# Tarea 4: interfaz conversacional (chat con los LLM de Llama 3 y Titan Premier)

En este cuaderno, se crea un chatbot con los modelos fundacionales (FM) llama3-8b-instruct y titan-text-premier en Amazon Bedrock.

Las interfaces conversacionales, como los chatbots y los asistentes virtuales, pueden mejorar la experiencia de usuario para sus clientes. Los chatbots usan algoritmos de machine learning y procesamiento de lenguaje natural (NLP) para comprender y responder las consultas de los usuarios. Puede usar chatbots en varias aplicaciones, como servicio al cliente, ventas y comercio electrónico, para ofrecer respuestas rápidas y eficientes a los usuarios. Los usuarios pueden acceder desde diferentes canales, como sitios web, plataformas de redes sociales y aplicaciones de mensajería.

- **Chatbot (básico)**: chatbot sin entrenamiento previo con un modelo fundacional.
- **Chatbot que usa una petición**: plantilla(LangChain): chatbot con cierto contexto en la plantilla de peticiones.
**Chatbot con personalidad**: chatbot con roles definidos, es decir, orientador de carrera profesional e interacciones humanas.
- **Chatbot consciente del contexto**: transferencia de contexto desde un archivo externo mediante la generación de incrustaciones.

## Marco de trabajo LangChain para crear un chatbot con Amazon Bedrock

En las interfaces conversacionales, como los chatbots, recordar las interacciones anteriores es muy importante, tanto a corto como a largo plazo.

El marco de trabajo LangChain proporciona componentes de memoria de dos formas. Primero, LangChain ofrece utilidades de ayuda para gestionar y manipular mensajes de chat anteriores. Se diseñaron para ser modulares. En segundo lugar, LangChain proporciona maneras sencillas de incorporar estas utilidades en cadenas, lo que le permite definir e interactuar de manera fácil con distintos tipos de abstracciones, que facilita la creación de chatbots eficaces.

## Crear un chatbot con contexto: elementos clave

El primer proceso para crear un chatbot consciente del contexto es generar incrustaciones para el contexto. En general, hay un proceso de ingesta que se ejecuta en el modelo de incrustación y genera las incrustaciones, que luego se almacenarán en un almacén de vectores. En este cuaderno, se utiliza el modelo Titan Embeddings para ello. El segundo proceso corresponde a la orquestación de la solicitud del usuario, la interacción, la invocación y la devolución de los resultados. Esto implica orquestar la solicitud del usuario, interactuar con los modelos o componentes necesarios para recopilar información, invocar al chatbot para formular una respuesta y, a continuación, devolver la respuesta del chatbot al usuario.

## Tarea 4.1: configuración del entorno

En esta tarea, establecerá el entorno.

In [ ]:
#ignore warnings and create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))


In [ ]:
# format instructions into a conversational prompt
from typing import Dict, List

def format_instructions(instructions: List[Dict[str, str]]) -> List[str]:
    """Format instructions where conversation roles must alternate system/user/assistant/user/assistant/..."""
    prompt: List[str] = []
    for instruction in instructions:
        if instruction["role"] == "system":
            prompt.extend(["<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n", (instruction["content"]).strip(), " <|eot_id|>"])
        elif instruction["role"] == "user":
            prompt.extend(["<|start_header_id|>user<|end_header_id|>\n", (instruction["content"]).strip(), " <|eot_id|>"])
        else:
            raise ValueError(f"Invalid role: {instruction['role']}. Role must be either 'user' or 'system'.")
    prompt.extend(["<|start_header_id|>assistant<|end_header_id|>\n"])
    return "".join(prompt)

## Tarea 4.2: usar el historial de chat de LangChain para iniciar la conversación

En esta tarea, permite que el chatbot transmita el contexto conversacional a través de varias interacciones con los usuarios. Contar con una memoria conversacional es crucial para que los chatbots mantengan diálogos significativos y coherentes a lo largo del tiempo.

Las capacidades de memoria conversacional se implementan a partir de la clase InMemoryChatMessageHistory de LangChain. Este objeto almacena las conversaciones entre el usuario y el chatbot, y el historial está disponible para el agente del chatbot a fin de que pueda aprovechar el contexto de una conversación anterior.

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Nota:** Las salidas del modelo son no deterministas.

In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_aws import ChatBedrock

chat_model=ChatBedrock(
    model_id="meta.llama3-8b-instruct-v1:0" , 
    client=bedrock_client)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Answer the following questions as best you can."),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

history = InMemoryChatMessageHistory()


def get_history():
    return history


chain = prompt | chat_model | StrOutputParser()

wrapped_chain = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key="chat_history",
)
query="how are you?"
response=wrapped_chain.invoke({"input": query})
# Printing history to see the history being built out. 
print(history)
# For the rest of the conversation, the output will only include response

### Nuevas preguntas

El modelo respondió con un mensaje inicial. Ahora, hágale algunas preguntas.

In [ ]:
#new questions
instructions = [{"role": "user", "content": "Give me a few tips on how to start a new garden."}]
response=wrapped_chain.invoke({"input": format_instructions(instructions)})
print(response)

### Partir de las preguntas

Ahora, haga una pregunta sin mencionar la palabra jardín para ver si el modelo puede comprender la conversación anterior.

In [ ]:
# build on the questions
instructions = [{"role": "user", "content": "bugs"}]
response=wrapped_chain.invoke({"input": format_instructions(instructions)})
print(response)

#### Finalizar la conversación

In [ ]:
# finishing the conversation
instructions = [{"role": "user", "content": "That's all, thank you!"}]
response=wrapped_chain.invoke({"input": format_instructions(instructions)})
print(response)

## Tarea 4.3: chatbot que usa una plantilla de peticiones (LangChain)

En esta tarea, utilizará la PromptTemplate predeterminada que es responsable de la construcción de esta entrada. LangChain ofrece varias clases y funciones para facilitar la creación de peticiones y el trabajo con ellas.

In [ ]:
#  prompt for a conversational agent
def format_prompt(actor:str, input:str):
    formatted_prompt: List[str] = []
    if actor == "system":
        prompt_template="""<|begin_of_text|><|start_header_id|>{actor}<|end_header_id|>\n{input}<|eot_id|>"""
    elif actor == "user":
        prompt_template="""<|start_header_id|>{actor}<|end_header_id|>\n{input}<|eot_id|>"""
    else:
        raise ValueError(f"Invalid role: {actor}. Role must be either 'user' or 'system'.")   
    prompt = PromptTemplate.from_template(prompt_template)     
    formatted_prompt.extend(prompt.format(actor=actor,input=input))
    formatted_prompt.extend(["<|start_header_id|>assistant<|end_header_id|>\n"])
    return "".join(formatted_prompt)

In [ ]:
# chat user experience
import ipywidgets as ipw
from IPython.display import display, clear_output

class ChatUX:
    """ A chat UX using IPWidgets
    """
    def __init__(self, qa, retrievalChain = False):
        self.qa = qa
        self.name = None
        self.b=None
        self.retrievalChain = retrievalChain
        self.out = ipw.Output()


    def start_chat(self):
        print("Starting chat bot")
        display(self.out)
        self.chat(None)


    def chat(self, _):
        if self.name is None:
            prompt = ""
        else: 
            prompt = self.name.value
        if 'q' == prompt or 'quit' == prompt or 'Q' == prompt:
            with self.out:
                print("Thank you , that was a nice chat !!")
            return
        elif len(prompt) > 0:
            with self.out:
                thinking = ipw.Label(value="Thinking...")
                display(thinking)
                try:
                    if self.retrievalChain:
                        response = self.qa.invoke({"input": prompt})
                        result=response['answer']
                    else:
                        instructions = [{"role": "user", "content": prompt}]
                        #result = self.qa.invoke({'input': format_prompt("user",prompt)}) #, 'history':chat_history})
                        result = self.qa.invoke({"input": format_instructions(instructions)})
                except:
                    result = "No answer"
                thinking.value=""
                print(f"AI:{result}")
                self.name.disabled = True
                self.b.disabled = True
                self.name = None

        if self.name is None:
            with self.out:
                self.name = ipw.Text(description="You:", placeholder='q to quit')
                self.b = ipw.Button(description="Send")
                self.b.on_click(self.chat)
                display(ipw.Box(children=(self.name, self.b)))

A continuación, inicie un chat.

In [ ]:
# start chat
history = InMemoryChatMessageHistory() #reset chat history
chat = ChatUX(wrapped_chain)
chat.start_chat()

In [ ]:
print(history)

## Tarea 4.4: chatbot con persona

En esta tarea, el asistente de inteligencia artificial (IA) desempeña el rol de un instructor de carrera profesional. Puede informar al chatbot sobre su persona (o rol) mediante un mensaje del sistema. Siga aprovechando la clase InMemoryChatMessageHistory para mantener el contexto conversacional.

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", " You will be acting as a career coach. Your goal is to give career advice to users. For questions that are not career related, don't provide advice. Say, I don't know."),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

history = InMemoryChatMessageHistory() # reset history

chain = prompt | chat_model | StrOutputParser()

wrapped_chain = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key="career_chat_history",
)

response=wrapped_chain.invoke({"input": "What are the career options in AI?"})
print(response)

In [ ]:
response=wrapped_chain.invoke({"input": "How to fix my car?"})
print(response)

In [ ]:
print(history)

Ahora, haga una pregunta que no forme parte de la especialidad de esta persona. El modelo no debe responder a esa pregunta y debe dar un motivo para ello.

## Tarea 4.5: chatbot con contexto

En esta tarea, pide al chatbot que responda a las preguntas en función del contexto que se le transfirió. Toma un archivo CSV y usa el modelo Titan Embeddings para crear un vector que represente ese contexto. Este vector se almacena en Facebook AI Similarity Search (FAISS). Cuando se le haga una pregunta al chatbot, usted pasará este vector al chatbot y hará que recupere la respuesta mediante el vector.

### Modelo Titan Embeddings

Las incrustaciones representan palabras, frases o cualquier otro elemento discreto como vectores en un espacio vectorial continuo. Esto permite que los modelos de machine learning realicen operaciones matemáticas a partir de estas representaciones y que recopilen las relaciones semánticas entre ellas.

Las incrustaciones se utilizan para la [capacidad de búsqueda en el documento](https://labelbox.com/blog/how-vector-similarity-search-works/) de la generación aumentada por recuperación (RAG).

In [ ]:
# model configuration
from langchain_aws.embeddings import BedrockEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate

br_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=bedrock_client)

### FAISS como VectorStore

Para usar incrustaciones en la búsqueda, es necesario contar con un almacén que pueda hacer con eficiencia búsquedas de similitudes de vectores. En este cuaderno, se usa FAISS, que es un almacenamiento que se encuentra en la memoria. Para almacenar vectores de manera permanente, puede usar las bases de conocimientos para Amazon Bedrock, pgVector, Pinecone, Weaviate o Chroma.

Las API de LangChain VectorStore están disponibles [aquí](https://python.langchain.com/v0.2/docs/integrations/vectorstores/).

In [ ]:
# vector store
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

loader = CSVLoader("../rag_data/Amazon_SageMaker_FAQs.csv") # --- > 219 docs with 400 chars
documents_aws = loader.load() #
print(f"documents:loaded:size={len(documents_aws)}")

docs = CharacterTextSplitter(chunk_size=2000, chunk_overlap=400, separator=",").split_documents(documents_aws)

print(f"Documents:after split and chunking size={len(docs)}")
vectorstore_faiss_aws = None
try:
    
    vectorstore_faiss_aws = FAISS.from_documents(
        documents=docs,
        embedding = br_embeddings, 
        #**k_args
    )

    print(f"vectorstore_faiss_aws:created={vectorstore_faiss_aws}::")

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

#### Ejecutar una prueba rápida de bajo código 

Se puede usar una clase Wrapper de LangChain para hacer una consulta al almacén de base de datos de vectores y obtener los documentos relevantes. Esto ejecuta una cadena de QA con todos los valores predeterminados.

In [ ]:
chat_llm=ChatBedrock(
    model_id="amazon.titan-text-premier-v1:0" , 
    client=bedrock_client)
# wrapper store faiss
wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss_aws)
print(wrapper_store_faiss.query("R in SageMaker", llm=chat_llm))

### Aplicación del chatbot

Para el chatbot, se necesita administración del contexto, historial, almacenes de vectores y muchos otros componentes. Empieza por crear una cadena de generación aumentada de recuperación (Retrieval Augmented Generation, RAG) que respalda el contexto.

Aquí, se usan las funciones **create_stuff_documents_chain** y **create_retrieval_chain**.

### Parámetros y funciones que se utilizan para la RAG

- **Retriever** (Recuperador): usó `VectorStoreRetriever`, que cuenta con el respaldo de `VectorStore`. Para recuperar texto, puede elegir entre dos tipos de búsqueda: `"similarity"` o `"mmr"`. `search_type="similarity"` usa la búsqueda de similitudes en el objeto recuperador, a partir del que selecciona los vectores de fragmentos de texto más similares al vector de la pregunta.

- **create_stuff_documents_chain** especifica cómo se introduce el contexto recuperado en una petición y un modelo de lenguaje grande (large language model, LLM). Los documentos recuperados se rellenan como contexto sin ningún resumen u otro procesamiento en la petición.

- **create_retrieval_chain** agrega el paso de recuperación y propaga el contexto recuperado a través de la cadena, a fin de proporcionarlo con la respuesta final. 

Si la pregunta formulada queda fuera del alcance del contexto, el modelo responderá que no sabe la respuesta.

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

retriever=vectorstore_faiss_aws.as_retriever()
question_answer_chain = create_stuff_documents_chain(chat_llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "What is sagemaker?"})
print(response) # shows the document chunks consulted to come up with the answer

A continuación, inicie un chat.

In [ ]:
chat = ChatUX(rag_chain, retrievalChain=True)
chat.start_chat()  # Only answers will be shown here, and not the citations


Se usó el LLM Titan para crear una interfaz conversacional con los siguientes patrones:

- Chatbot (básico: sin contexto)
- Chatbot que usa una plantilla de peticiones (Langchain)
- Chatbot con personas
- Chatbot con contexto

### Pruébelo usted mismo

- Cambie las peticiones para que se adapten al caso práctico específico y evalúe la salida de los diferentes modelos.
- Juegue con la longitud del token para comprender la latencia y la capacidad de respuesta del servicio.
- Aplique diferentes principios de ingeniería de peticiones para obtener mejores salidas.

### Limpieza

Ha completado este cuaderno. Para ir a la siguiente parte del laboratorio, complete estos pasos:

- Cierre este archivo de cuaderno y continúe con la **Tarea 5**.